In [3]:
import pandas as pd
import pickle

In [21]:
url_test = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv"
df_test = pd.read_csv(url_test)

In [22]:
df_test.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,1,1,PAIDOFF,1000,30,9/8/2016,10/7/2016,50,Bechalor,female
1,5,5,PAIDOFF,300,7,9/9/2016,9/15/2016,35,Master or Above,male
2,21,21,PAIDOFF,1000,30,9/10/2016,10/9/2016,43,High School or Below,female
3,24,24,PAIDOFF,1000,30,9/10/2016,10/9/2016,26,college,male
4,35,35,PAIDOFF,800,15,9/11/2016,9/25/2016,29,Bechalor,male


In [23]:
def dayofweek(df):
    df['due_date'] = pd.to_datetime(df['due_date'])
    df['effective_date'] = pd.to_datetime(df['effective_date'])
    df['dayofweek'] = df['effective_date'].dt.dayofweek
    df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
    return df

def cat_cols(df):
    df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
    Feature = df[['Principal','terms','age','Gender','weekend']]
    Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
    Feature.drop(['Master or Above'], axis = 1,inplace=True)
    return Feature

In [26]:
def copy_df(test_df):
   return test_df.copy()
   
data_clean = (df_test.
                pipe(copy_df).
                pipe(dayofweek).
                pipe(cat_cols))
y_test = df_test["loan_status"]

In [29]:
from sklearn import svm
import bentoml.sklearn

rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo',probability=True).fit(data_clean, y_test)


bentoml.sklearn.save("Loan_Status_SVM", rbf)


[20:30:51] WARNING  converting Loan_Status_SVM to lowercase: loan_status_svm

           INFO     Successfully saved Model(tag="loan_status_svm:b6k4unute6434af3",         
                    path="C:\Users\sasha\bentoml\models\loan_status_svm\b6k4unute6434af3\")

Tag(name='loan_status_svm', version='b6k4unute6434af3')